# Testing
---
This notebook will be my testing method until one test becomes too big to handle

In [11]:
from qiskit.quantum_info import Pauli,SparsePauliOp,Statevector

In [16]:
psi = Statevector.from_int(4,dims=8)
psi

Statevector([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j],
            dims=(2, 2, 2))


In [8]:
pl = Pauli('ZZI')

print(pl.to_matrix(sparse=True))

  (0, 0)	(1+0j)
  (1, 1)	(1+0j)
  (2, 2)	(-1+0j)
  (3, 3)	(-1+0j)
  (4, 4)	(-1+0j)
  (5, 5)	(-1+0j)
  (6, 6)	(1+0j)
  (7, 7)	(1+0j)


In [10]:
H_cut = SparsePauliOp(Pauli('ZZI'))+ SparsePauliOp(Pauli('ZIZ'))

print("H_cut is")
print(H_cut)
print("The sparse representation of H_cut is")
print(H_cut.to_matrix(sparse=True))

H_cut is
SparsePauliOp(['ZZI', 'ZIZ'],
              coeffs=[1.+0.j, 1.+0.j])
The sparse representation of H_cut is
  (0, 0)	(2+0j)
  (1, 1)	0j
  (2, 2)	0j
  (3, 3)	(-2+0j)
  (4, 4)	(-2+0j)
  (5, 5)	0j
  (6, 6)	0j
  (7, 7)	(2+0j)


In [18]:
psi.expectation_value(H_cut).real

-2.0